In [1]:
import pickle
import pandas as pd
import re


with open('data_pickle_format/synethetictotrain.pickle', 'rb') as f:
    synethetic = pickle.load(f)
with open('data_pickle_format/labeledtotrain.pickle', 'rb') as f:
    labeled = pickle.load(f)
with open('data_pickle_format/unlabeled_vacancies_final.pickle', 'rb') as f:
    unlabeled = pickle.load(f)

In [2]:
from string import punctuation
import html
import re

def preprocess_text(text):
    text = text.lower()
    text = html.unescape(text)
    text = re.sub(r"https?://\S+", "", text) # remove http links
    text = re.sub(r"<.*?>", " ", text) # remove html tags
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text

unlabeled["job_description"] = unlabeled["job_description"].map(preprocess_text)

pd.set_option('display.max_colwidth', None)

In [3]:
training = pd.concat([synethetic, labeled], axis=0)

In [4]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

train_df = training[["job_description", "women_proportion"]].dropna()
train_df.columns = ["text", "labels"]

from sklearn.model_selection import train_test_split
train_df, dev_df = train_test_split(train_df, test_size=0.1, random_state=42)


/home/jkoz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.num_train_epochs = 2
model_args.evaluate_during_training = True
model_args.regression = True   
model_args.train_batch_size = 8 
model_args.eval_batch_size = 8
model_args.max_seq_length = 128 

model = ClassificationModel(
    "distilbert",
    "distilbert-base-multilingual-cased",
    num_labels=1,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

model.train_model(train_df[['text', 'labels']].sample(100), eval_df=dev_df[['text', 'labels']].sample(20))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
1it [00:00,  4.39it/s]
Epochs 1/2. Running Loss:    0.1975: 100%|██████████| 13/13 [00:28<00:00,  2.16s/it]
1it [00:00,  4.81it/s]


In [ ]:

# preds, raw_outputs = model.predict(unlabeled['job_description'].tolist())